In [1]:
!nvidia-smi

Sat May 01 21:00:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 441.93       Driver Version: 441.93       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1050   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   38C    P8    N/A /  N/A |    169MiB /  4096MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [3]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [4]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]
train_path = "C:/Users/Vijaya/Tomato_plant/raw/color/train/"
valid_path = "C:/Users/Vijaya/Tomato_plant/raw/color/test/"

In [5]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
import tensorflow
# add preprocessing layer to the front of VGG
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [6]:
# don't train existing weights
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [7]:
# useful for getting number of output classes
folders = glob('C:/Users/Vijaya/Tomato_plant/raw/color/train/*')

In [8]:
folders

['C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Bacterial_spot',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Early_blight',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___healthy',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Late_blight',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Leaf_Mold',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Septoria_leaf_spot',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Spider_mites Two-spotted_spider_mite',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Target_Spot',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Tomato_mosaic_virus',
 'C:/Users/Vijaya/Tomato_plant/raw/color/train\\Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [9]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)


In [10]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [11]:
# view the structure of the model
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [12]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [13]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory("C:/Users/Vijaya/Tomato_plant/raw/color/train/",
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 14496 images belonging to 10 classes.


In [15]:
test_set = test_datagen.flow_from_directory("C:/Users/Vijaya/Tomato_plant/raw/color/test/",
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 3664 images belonging to 10 classes.


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
906/906 [==============================] - 3122s 3s/step - loss: 0.8487 - accuracy: 0.7490 - val_loss: 0.9701 - val_accuracy: 0.7765
Epoch 2/50
906/906 [==============================] - 2555s 3s/step - loss: 0.6080 - accuracy: 0.8280 - val_loss: 0.6730 - val_accuracy: 0.8507
Epoch 3/50
906/906 [==============================] - 2573s 3s/step - loss: 0.4854 - accuracy: 0.8633 - val_loss: 0.8059 - val_accuracy: 0.8362
Epoch 4/50
906/906 [==============================] - 2470s 3s/step - loss: 0.4857 - accuracy: 0.8736 - val_loss: 0.5438 - val_accuracy: 0.8654
Epoch 5/50
906/906 [==============================] - 2467s 3s/step - loss: 0.3756 - accuracy: 0.8968 - val_loss: 0.5427 - val_accuracy: 0.8712
Epoch 6/50
906/906 [==============================] - 2462s 3s/step - loss: 0.3857 - accuracy: 0.8949 - val_loss: 0.8505 - val_accuracy: 0.8390
Epoch 7/50
906/906 [==============================] - 2467s 

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')